# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import os
import hvplot
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,holualoa,19.6228,-155.9522,23.64,89,85,1.83,US,1724825382
1,1,port-aux-francais,-49.3500,70.2167,5.40,97,100,13.80,TF,1724825384
2,2,waitangi,-43.9535,-176.5597,11.01,96,100,4.92,NZ,1724825385
3,3,puerto natales,-51.7236,-72.4875,3.84,82,91,6.39,CL,1724825386
4,4,sena madureira,-9.0656,-68.6569,20.93,57,1,0.49,BR,1724825387


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
city_data_df = city_data_df.dropna(subset=['Lat', 'Lng'])
map_plot = city_data_df.hvplot.points(
    'Lng', 'Lat',
    geo=True,
    tiles='OSM',  
    size=20,
    color='City'
).opts(
    xlabel="x",
    ylabel="y",
    legend_position="right",
    frame_height=500, 
    frame_width=800
)
# Display the map
map_plot


:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City)

In [4]:
directory = 'vacationpyhvplots/'
file_path = os.path.join(directory, 'map_plot.png')

# Create the directory if it does not exist
os.makedirs(directory, exist_ok=True)

# Save the map plot as a PNG file
hvplot.save(map_plot, file_path, fmt='png')

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [5]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_temp_min = 21   
ideal_temp_max = 27  
ideal_cloudiness = 0  
ideal_wind_speed_max = 4.5  

ideal_weather_cities_df = city_data_df.loc[
    (city_data_df['Max Temp'] >= ideal_temp_min) & 
    (city_data_df['Max Temp'] <= ideal_temp_max) &
    (city_data_df['Cloudiness'] == ideal_cloudiness) &
    (city_data_df['Wind Speed'] <= ideal_wind_speed_max)
]

# Drop any rows with null values
ideal_weather_cities_df = ideal_weather_cities_df.dropna()

# Display sample data
ideal_weather_cities_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
5,5,mozarlandia,-14.7447,-50.5706,25.56,30,0,3.85,BR,1724825388
81,81,savannah,32.0835,-81.0998,26.08,88,0,0.00,US,1724825262
89,89,jamestown,42.0970,-79.2353,21.80,97,0,1.54,US,1724825489
102,102,hamilton,39.1834,-84.5333,26.10,79,0,0.00,US,1724825504
149,149,palapye,-22.5460,27.1251,21.83,17,0,1.85,BW,1724825566
218,218,turaif,31.6725,38.6637,26.14,36,0,3.09,SA,1724825650
223,223,columbus,39.9612,-82.9988,25.32,78,0,2.59,US,1724825427
252,252,canutama,-6.5339,-64.3831,24.19,49,0,1.13,BR,1724825694
298,298,santana,-0.0583,-51.1817,26.10,89,0,1.03,BR,1724825753
306,306,dondo,-19.6094,34.7431,25.59,60,0,3.32,MZ,1724825763


### Step 3: Create a new DataFrame called `hotel_df`.

In [6]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_weather_cities_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
5,mozarlandia,BR,-14.7447,-50.5706,30,
81,savannah,US,32.0835,-81.0998,88,
89,jamestown,US,42.0970,-79.2353,97,
102,hamilton,US,39.1834,-84.5333,79,
149,palapye,BW,-22.5460,27.1251,17,
218,turaif,SA,31.6725,38.6637,36,
223,columbus,US,39.9612,-82.9988,78,
252,canutama,BR,-6.5339,-64.3831,49,
298,santana,BR,-0.0583,-51.1817,89,
306,dondo,MZ,-19.6094,34.7431,60,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [7]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories": "accommodation.hotel", 
    "limit": 1, 
    "apiKey": geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row['Lat']
    longitude = row['Lng']

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
mozarlandia - nearest hotel: No hotel found
savannah - nearest hotel: Comfort Inn
jamestown - nearest hotel: DoubleTree Jamestown
hamilton - nearest hotel: North Vista Manor
palapye - nearest hotel: Palapaye Hotel
turaif - nearest hotel: المودة
columbus - nearest hotel: Sheraton Columbus Hotel at Capitol Square
canutama - nearest hotel: No hotel found
santana - nearest hotel: Hotel Teodoro
dondo - nearest hotel: No hotel found
fort bragg - nearest hotel: Airborne Inn Lodging
preobrazheniye - nearest hotel: No hotel found
ivins - nearest hotel: Crescent Moon Inn
maraba - nearest hotel: Portobelo
kill devil hills - nearest hotel: Mariner Days Inn & Suites
page - nearest hotel: Clarion Inn Page - Lake Powell
kashgar - nearest hotel: 深航国际酒店
abilene - nearest hotel: DoubleTree by Hilton Abilene Downtown Convention Center
unai - nearest hotel: Hotel Fazenda Curva do Rio


,City,Country,Lat,Lng,Humidity,Hotel Name
5,mozarlandia,BR,-14.7447,-50.5706,30,No hotel found
81,savannah,US,32.0835,-81.0998,88,Comfort Inn
89,jamestown,US,42.0970,-79.2353,97,DoubleTree Jamestown
102,hamilton,US,39.1834,-84.5333,79,North Vista Manor
149,palapye,BW,-22.5460,27.1251,17,Palapaye Hotel
218,turaif,SA,31.6725,38.6637,36,المودة
223,columbus,US,39.9612,-82.9988,78,Sheraton Columbus Hotel at Capitol Square
252,canutama,BR,-6.5339,-64.3831,49,No hotel found
298,santana,BR,-0.0583,-51.1817,89,Hotel Teodoro
306,dondo,MZ,-19.6094,34.7431,60,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [8]:
%%capture --no-display

# Configure the map plot
map_plot_with_hotels = hotel_df.hvplot.points(
    'Lng', 'Lat',
    geo=True,
    tiles='OSM',  
    size=55,
    color='City',
    hover_cols=['Lng', 'Lat', 'City', 'Humidity', 'Hotel Name', 'Country'],
    frame_height=500, 
    frame_width=800
).opts(
    xlabel="x",
    ylabel="y",
    legend_position="right"
)
# Display the map
map_plot_with_hotels

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)

In [9]:
directory = 'vacationpyhvplots/'
file_path = os.path.join(directory, 'map_plot_with_hotels.png')

# Create the directory if it does not exist
os.makedirs(directory, exist_ok=True)

# Save the map plot as a PNG file
hvplot.save(map_plot_with_hotels, file_path, fmt='png')